In [193]:
import re
import random
import string

In [194]:
import pandas as pd
train_data = pd.read_csv('/Users/lemarx/Documents/01_projects/SentencesRelatedness24/data/raw/eng_train.csv')
tada = train_data['Text'].loc[0]
tada

'It that happens, just pull the plug.\nif that ever happens, just pull the plug.'

In [195]:
def get_random_word(seq : str,sep :str = '[SEP]', min_len : int = 3,seed : int = 42) -> str:
    #random.seed(seed)
    seq = seq.replace(sep,'')
    seq = re.sub(r"[^a-zA-Z0-9]+", ' ', seq)
    seq = list(set(seq.split()))
    candidates = [word for word in seq if len(word) >= min_len]

    if(candidates == []):
        return None

    return random.choice(candidates)


In [196]:
#changes one random letter of the word
#dont interchange first or last letter
def replace_letter(word : str, seed : int = 42)-> str:
    #random.seed(42)
    if len(word) <=2:
        return word
    idx = random.randint(1,len(word)-2)
    mod_word = word[:idx] + random.choice(string.ascii_lowercase) + word[idx + 1:] 
    return mod_word


In [274]:
def apply_change_letter(seq : str, p : int = 0.3, sep :str = '[SEP]'):
    seq = seq.split(sep)
    word = get_random_word(seq[0],sep)
    if word == None or random.random() < p:
        return seq[0] + sep + seq[1]
    return seq[0].replace(word,replace_letter(word)) + sep + seq[1]

In [283]:
print(apply_change_letter(tada,sep = '\n'))

It thnt happens, just pull the plug.
if that ever happens, just pull the plug.
